# Importing Tensorflow

In [ ]:

!pip install tensorflow
# MUST restart runtime after running this 

In [ ]:
# checking tensorflow version
import tensorflow as tf
tf.random.set_seed(424)
# from tensorflow.contrib.tensor_forest import tensor_forest
# from tensorflow.python.ops import resources
print(tf.__version__)

2.4.1


In [1]:
# Import libraries
from __future__ import print_function
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Binary Classification with Sonar Dataset: Baseline
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import keras
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
# normalize [-1,1]
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = 2*((df[feature_name] - min_value) / (max_value - min_value))-1
    return result

# normalize [0,1] - not using 
def normalize01(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = 2*((df[feature_name] - min_value) / (max_value - min_value))-1
    return result

In [3]:
sheet_name = ['BSE_Sensex', 'NIFTY_50', 'Reliance', 'Infosys']
BSE_Sensex_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[0])
# NIFTY_50_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[1])
# Reliance_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[2])
# Infosys_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[3])

# **ANN**

In [4]:
BSE_Sensex_feature

,Date,Close,MA_10,WMA_10,Momentum_10,SO%k,SO%d_10,RSI_10,MACD,LW_R,AD_R,CCI_10,label
0,1991-03-14,1196.48,1196.133,1448.361111,-17.79,0.198185,0.612414,-613.827993,36.722136,-80.181530,0.798127,-56.850448,1
1,1991-03-15,1206.75,1195.261,1448.438222,-8.72,0.518946,0.570987,-1356.307339,35.104788,-48.105393,0.792338,-23.627201,1
2,1991-03-18,1171.08,1193.781,1450.991333,-14.80,0.609447,0.531932,-758.040541,33.875538,-39.055340,1.327092,56.462264,0
3,1991-03-20,1171.92,1188.848,1445.946667,-49.33,0.295118,0.473500,-227.427529,32.335611,-70.488192,-1.097561,-26.504612,1
4,1991-03-21,1165.20,1183.496,1442.184889,-53.52,0.302520,0.414585,-201.793722,30.627530,-69.747973,0.849635,-25.001604,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7233,2020-12-24,46973.54,46309.784,56628.973778,340.68,0.713257,0.814947,555.075144,1021.280189,-28.674254,0.824835,8.552392,1
7234,2020-12-28,47353.75,46411.150,56776.475111,1013.66,0.961479,0.819005,238.777302,1001.588984,-3.852087,1.184377,95.661306,1
7235,2020-12-29,47613.08,46536.624,56985.941778,1254.74,0.978672,0.825973,212.113266,990.237684,-2.132757,1.675874,131.145622,1
7236,2020-12-30,47746.22,46672.586,57225.154222,1359.62,0.963650,0.827654,203.464939,986.494748,-3.635002,1.023111,132.583835,1


# Data preparation

In [18]:
from pandas import read_csv
# from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
data = BSE_Sensex_feature.dropna() # DROPNA

del data["Close"]

#Extract feature and target np arrays (inputs for placeholders)
input_x = normalize(data.iloc[:, 1:-1]).values  # NORMALIZE 
input_y = data.iloc[:, -1:].values

# Splitting the dataset into the Training set and Test set

#X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size = 0.25, random_state = 0)


In [19]:
# Construct a time series input features
# as defined by how many prior days of input features to be used to predict today's stock movement
num_days = 3
new_input_x = []
for i in range (num_days, input_x.shape[0]+1):
    new_input_x.append(input_x[i-num_days: i, :].flatten())
new_input_x = np.array(new_input_x)

new_input_y = input_y[num_days-1:]

# X_train = new_input_x[:int(new_input_x.shape[0]*0.8), :]
# y_train = new_input_y[:int(new_input_y.shape[0]*0.8), :]
# X_test = new_input_x[int(new_input_x.shape[0]*0.8):, :]
# y_test = new_input_y[int(new_input_y.shape[0]*0.8):, :]

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(new_input_x, new_input_y, test_size = 0.2, random_state = 424)


In [24]:
new_input_x.shape

(7236, 30)

In [23]:
print("percentage of price UP in FULL DATASET: {}".format((BSE_Sensex_feature['label']==1).sum()/BSE_Sensex_feature.shape[0]))
print("percentage of price UP in TRAINING DATAASET: {}".format((y_train==1).sum()/y_train.shape[0]))
print("percentage of price UP in TEST DATAASET: {}".format((y_test==1).sum()/y_test.shape[0]))

percentage of price UP in FULL DATASET: 0.5316385741917656
percentage of price UP in TRAINING DATAASET: 0.5295438838977194
percentage of price UP in TEST DATAASET: 0.539364640883978


# Adam

## batch 32

In [ ]:
%%time
tf.random.set_seed(424)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.1)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100,validation_split=0.1)
# mlp__validation_split=0.3

Epoch 1/100
163/163 [==============================] - 1s 4ms/step - loss: 1.2101 - accuracy: 0.5177 - val_loss: 0.6892 - val_accuracy: 0.5630
Epoch 2/100
163/163 [==============================] - 0s 3ms/step - loss: 0.6924 - accuracy: 0.5151 - val_loss: 0.7044 - val_accuracy: 0.4370
Epoch 3/100
163/163 [==============================] - 0s 3ms/step - loss: 0.6936 - accuracy: 0.5131 - val_loss: 0.6946 - val_accuracy: 0.4370
Epoch 4/100
163/163 [==============================] - 0s 3ms/step - loss: 0.6933 - accuracy: 0.5175 - val_loss: 0.6852 - val_accuracy: 0.5630
Epoch 5/100
163/163 [==============================] - 0s 3ms/step - loss: 0.6930 - accuracy: 0.5256 - val_loss: 0.6852 - val_accuracy: 0.5630
Epoch 6/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.5186 - val_loss: 0.6884 - val_accuracy: 0.5630
Epoch 7/100
163/163 [==============================] - 0s 3ms/step - loss: 0.6940 - accuracy: 0.5235 - val_loss: 0.6880 - val_accuracy: 0.5630

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.6905 - accuracy: 0.5394
Accuracy: 53.94


## batch 64

In [ ]:
%%time
tf.random.set_seed(424)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.1)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size = 64, validation_split=0.1)
# mlp__validation_split=0.3

Epoch 1/100
82/82 [==============================] - 1s 6ms/step - loss: 3.6060 - accuracy: 0.4932 - val_loss: 0.6903 - val_accuracy: 0.5630
Epoch 2/100
82/82 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.5100 - val_loss: 0.6895 - val_accuracy: 0.5630
Epoch 3/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6925 - accuracy: 0.5285 - val_loss: 0.6855 - val_accuracy: 0.5630
Epoch 4/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6964 - accuracy: 0.4956 - val_loss: 0.6853 - val_accuracy: 0.5630
Epoch 5/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6924 - accuracy: 0.5215 - val_loss: 0.6856 - val_accuracy: 0.5630
Epoch 6/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6928 - accuracy: 0.5215 - val_loss: 0.6886 - val_accuracy: 0.5630
Epoch 7/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.5260 - val_loss: 0.6865 - val_accuracy: 0.5630
Epoch 8/100
8

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.6908 - accuracy: 0.5394
Accuracy: 53.94


# Adadelta

## batch 32

In [ ]:
%%time
tf.random.set_seed(424)
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.Adadelta(
    learning_rate=0.1, rho=0.95, epsilon=1e-07, name="Adadelta"
        )
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, validation_split=0.1)
# mlp__validation_split=0.3

current days:  3
Epoch 1/100
163/163 [==============================] - 1s 3ms/step - loss: 0.6937 - accuracy: 0.5014 - val_loss: 0.6875 - val_accuracy: 0.5751
Epoch 2/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6899 - accuracy: 0.5319 - val_loss: 0.6854 - val_accuracy: 0.5820
Epoch 3/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6870 - accuracy: 0.5507 - val_loss: 0.6842 - val_accuracy: 0.5786
Epoch 4/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6895 - accuracy: 0.5370 - val_loss: 0.6814 - val_accuracy: 0.5751
Epoch 5/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6850 - accuracy: 0.5541 - val_loss: 0.6814 - val_accuracy: 0.5803
Epoch 6/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6871 - accuracy: 0.5443 - val_loss: 0.6838 - val_accuracy: 0.5769
Epoch 7/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6856 - accuracy: 0.5592 - val_loss: 0.6811 - val

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.6994 - accuracy: 0.5463
Accuracy: 54.63


## batch 64

In [ ]:
%%time
tf.random.set_seed(424)
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.Adadelta(
    learning_rate=0.1, rho=0.95, epsilon=1e-07, name="Adadelta"
        )
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100,batch_size = 64, validation_split=0.1)
# mlp__validation_split=0.3

current days:  3
Epoch 1/100
82/82 [==============================] - 1s 5ms/step - loss: 0.6943 - accuracy: 0.4906 - val_loss: 0.6889 - val_accuracy: 0.5682
Epoch 2/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6908 - accuracy: 0.5298 - val_loss: 0.6859 - val_accuracy: 0.5699
Epoch 3/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6879 - accuracy: 0.5410 - val_loss: 0.6850 - val_accuracy: 0.5751
Epoch 4/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6903 - accuracy: 0.5269 - val_loss: 0.6829 - val_accuracy: 0.5734
Epoch 5/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6865 - accuracy: 0.5479 - val_loss: 0.6827 - val_accuracy: 0.5786
Epoch 6/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6879 - accuracy: 0.5399 - val_loss: 0.6846 - val_accuracy: 0.5820
Epoch 7/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6865 - accuracy: 0.5524 - val_loss: 0.6827 - val_accuracy: 0.5

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.6940 - accuracy: 0.5470
Accuracy: 54.70


# SGD

## batch 32

In [ ]:
%%time
tf.random.set_seed(424)
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.SGD(learning_rate=0.1, nesterov=False, name="SGD")
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100,validation_split=0.1)
# mlp__validation_split=0.3

current days:  3
Epoch 1/100
163/163 [==============================] - 1s 3ms/step - loss: 0.6938 - accuracy: 0.5039 - val_loss: 0.6868 - val_accuracy: 0.5803
Epoch 2/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6887 - accuracy: 0.5426 - val_loss: 0.6865 - val_accuracy: 0.5751
Epoch 3/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6862 - accuracy: 0.5576 - val_loss: 0.6813 - val_accuracy: 0.5803
Epoch 4/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6891 - accuracy: 0.5427 - val_loss: 0.6790 - val_accuracy: 0.5941
Epoch 5/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6824 - accuracy: 0.5517 - val_loss: 0.6786 - val_accuracy: 0.5993
Epoch 6/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6861 - accuracy: 0.5528 - val_loss: 0.6843 - val_accuracy: 0.5630
Epoch 7/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6846 - accuracy: 0.5615 - val_loss: 0.6784 - val

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.7296 - accuracy: 0.5242
Accuracy: 52.42


In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.7322 - accuracy: 0.5394
Accuracy: 53.94


## batch 64

In [ ]:
%%time
tf.random.set_seed(424)
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.SGD(
    learning_rate=0.1, momentum=0.1, nesterov=False, name="SGD"
)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64,validation_split=0.1)

# mlp__validation_split=0.3

current days:  3
Epoch 1/100
82/82 [==============================] - 1s 5ms/step - loss: 0.6935 - accuracy: 0.4987 - val_loss: 0.6877 - val_accuracy: 0.5924
Epoch 2/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6892 - accuracy: 0.5396 - val_loss: 0.6857 - val_accuracy: 0.5820
Epoch 3/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6869 - accuracy: 0.5483 - val_loss: 0.6822 - val_accuracy: 0.5682
Epoch 4/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6901 - accuracy: 0.5321 - val_loss: 0.6805 - val_accuracy: 0.5855
Epoch 5/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6844 - accuracy: 0.5529 - val_loss: 0.6800 - val_accuracy: 0.5907
Epoch 6/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6871 - accuracy: 0.5496 - val_loss: 0.6862 - val_accuracy: 0.5717
Epoch 7/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6860 - accuracy: 0.5526 - val_loss: 0.6797 - val_accuracy: 0.5

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.7218 - accuracy: 0.5325
Accuracy: 53.25


# Adagrad

## batch 32

In [ ]:
%%time
tf.random.set_seed(424)
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.Adagrad(
    learning_rate=0.1,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad"
)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100,validation_split=0.1)
# mlp__validation_split=0.3

current days:  3
Epoch 1/100
163/163 [==============================] - 1s 4ms/step - loss: 0.6944 - accuracy: 0.5215 - val_loss: 0.6887 - val_accuracy: 0.6028
Epoch 2/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6892 - accuracy: 0.5391 - val_loss: 0.6855 - val_accuracy: 0.5941
Epoch 3/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6858 - accuracy: 0.5598 - val_loss: 0.6790 - val_accuracy: 0.5769
Epoch 4/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6895 - accuracy: 0.5401 - val_loss: 0.6799 - val_accuracy: 0.5924
Epoch 5/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.5544 - val_loss: 0.6793 - val_accuracy: 0.6010
Epoch 6/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6868 - accuracy: 0.5545 - val_loss: 0.6848 - val_accuracy: 0.5630
Epoch 7/100
163/163 [==============================] - 0s 2ms/step - loss: 0.6847 - accuracy: 0.5533 - val_loss: 0.6788 - val

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.9185 - accuracy: 0.5421
Accuracy: 54.21


## batch 64

In [ ]:
%%time
tf.random.set_seed(424)
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(60, input_dim=10*num_days, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.Adagrad(
    learning_rate=0.1,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad"
)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100,batch_size = 64, validation_split=0.1)
# mlp__validation_split=0.3

current days:  3
Epoch 1/100
82/82 [==============================] - 1s 5ms/step - loss: 0.6940 - accuracy: 0.5133 - val_loss: 0.6870 - val_accuracy: 0.5820
Epoch 2/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6890 - accuracy: 0.5405 - val_loss: 0.6855 - val_accuracy: 0.5803
Epoch 3/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6858 - accuracy: 0.5616 - val_loss: 0.6806 - val_accuracy: 0.5838
Epoch 4/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6894 - accuracy: 0.5407 - val_loss: 0.6801 - val_accuracy: 0.6045
Epoch 5/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6828 - accuracy: 0.5644 - val_loss: 0.6787 - val_accuracy: 0.5993
Epoch 6/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6864 - accuracy: 0.5528 - val_loss: 0.6842 - val_accuracy: 0.5699
Epoch 7/100
82/82 [==============================] - 0s 3ms/step - loss: 0.6852 - accuracy: 0.5555 - val_loss: 0.6792 - val_accuracy: 0.5

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.7677 - accuracy: 0.5366
Accuracy: 53.66


# Some other testing topics

## heavy shallow ANN

In [ ]:
%%time
tf.random.set_seed(424)
num_days = 3
print("current days: ", num_days)
# define the keras model
model = Sequential()
model.add(Dense(336, input_dim=10*num_days, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.1)
opt = keras.optimizers.Adagrad(
    learning_rate=0.1,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad"
)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)
#model.fit(X_train, y_train, epochs=100,validation_split=0.1)
# mlp__validation_split=0.3

current days:  3
Epoch 1/100
181/181 [==============================] - 1s 1ms/step - loss: 0.6997 - accuracy: 0.5192
Epoch 2/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6903 - accuracy: 0.5431
Epoch 3/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6868 - accuracy: 0.5494
Epoch 4/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6847 - accuracy: 0.5521
Epoch 5/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6858 - accuracy: 0.5490
Epoch 6/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6840 - accuracy: 0.5607
Epoch 7/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6828 - accuracy: 0.5635
Epoch 8/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6853 - accuracy: 0.5511
Epoch 9/100
181/181 [==============================] - 0s 1ms/step - loss: 0.6822 - accuracy: 0.5713
Epoch 10/100
181/181 [==============================] - 0s 1ms/step - loss

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 2ms/step - loss: 0.7148 - accuracy: 0.5014
Accuracy: 50.14


## Orignal paper's 10 input features model

In [ ]:
# Run for the paper methodology, 10 input features
num_days = 1
new_input_x = []
for i in range (num_days, input_x.shape[0]+1):
    new_input_x.append(input_x[i-num_days: i, :].flatten())
new_input_x = np.array(new_input_x)

new_input_y = input_y[num_days-1:]

X_train, X_test, y_train, y_test = train_test_split(new_input_x, new_input_y, test_size = 0.2, random_state = 424)
# define the keras model
tf.random.set_seed(424)
model = Sequential()

model.add(Dense(50, input_dim=10*num_days, activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


opt = keras.optimizers.Adam(learning_rate=0.001)
# opt = keras.optimizers.Adadelta(
#     learning_rate=0.01, rho=0.95, epsilon=1e-07, name="Adadelta"
#         )
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)
# mlp__validation_split=0.3

Epoch 1/100
181/181 [==============================] - 1s 2ms/step - loss: 0.6905 - accuracy: 0.5309
Epoch 2/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6878 - accuracy: 0.5396
Epoch 3/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6889 - accuracy: 0.5392
Epoch 4/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6863 - accuracy: 0.5494
Epoch 5/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6870 - accuracy: 0.5518
Epoch 6/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6843 - accuracy: 0.5636
Epoch 7/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.5627
Epoch 8/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.5668
Epoch 9/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6851 - accuracy: 0.5686
Epoch 10/100
181/181 [==============================] - 0s 2ms/step - loss: 0.6861 - accura

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

46/46 [==============================] - 0s 1ms/step - loss: 0.8967 - accuracy: 0.5414
Accuracy: 54.14


## Find best number of years
Adagrad

In [ ]:
# Construct a time series input features
# as defined by how many prior days of input features to be used to predict today's stock movement
tf.random.set_seed(424)
for num_days in [4,5,6,7,8,9,10]:

    new_input_x = []
    for i in range (num_days, input_x.shape[0]+1):
        new_input_x.append(input_x[i-num_days: i, :].flatten())
    new_input_x = np.array(new_input_x)

    new_input_y = input_y[num_days-1:]

    # X_train = new_input_x[:int(new_input_x.shape[0]*0.8), :]
    # y_train = new_input_y[:int(new_input_y.shape[0]*0.8), :]
    # X_test = new_input_x[int(new_input_x.shape[0]*0.8):, :]
    # y_test = new_input_y[int(new_input_y.shape[0]*0.8):, :]

    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(new_input_x, new_input_y, test_size = 0.2, random_state = 424)

    
    print("current days: ", num_days)
    # define the keras model
    model = Sequential()
    model.add(Dense(100, input_dim=10*num_days, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # opt = keras.optimizers.Adam(learning_rate=0.1)
    opt = keras.optimizers.Adagrad(
        learning_rate=0.1,
        initial_accumulator_value=0.1,
        epsilon=1e-07,
        name="Adagrad"
    )

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    training_history = model.fit(X_train, y_train, epochs=100, verbose=0, validation_split=0.1)
    
    _, accuracy = model.evaluate(X_test, y_test)
    print(training_history)
    print('Accuracy: %.2f' % (accuracy*100))


current days:  4
46/46 [==============================] - 0s 1ms/step - loss: 1.2638 - accuracy: 0.5328
Accuracy: 53.28
current days:  5
46/46 [==============================] - 0s 1ms/step - loss: 1.4677 - accuracy: 0.5266
Accuracy: 52.66
current days:  6
46/46 [==============================] - 0s 1ms/step - loss: 1.6303 - accuracy: 0.5294
Accuracy: 52.94
current days:  7
46/46 [==============================] - 0s 1ms/step - loss: 2.2625 - accuracy: 0.5100
Accuracy: 51.00
current days:  8
46/46 [==============================] - 0s 1ms/step - loss: 2.4121 - accuracy: 0.5121
Accuracy: 51.21
current days:  9
46/46 [==============================] - 0s 1ms/step - loss: 2.8318 - accuracy: 0.5159
Accuracy: 51.59
current days:  10
46/46 [==============================] - 0s 1ms/step - loss: 2.2259 - accuracy: 0.5097
Accuracy: 50.97
